In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sys; 
#sys.path.append('/home/mitch/school/data/mexico_enigh/src/')
sys.path.append('../src')
sys.path.append('/home/mitch/school/data/mexico_enigh/dicts/')
sys.path.append('/home/mitch/util/python/')
import json_utils

import utils
import os

from simpledbf import Dbf5

In [54]:
raw  = '/home/mitch/Dropbox/data/mexico_enigh/raw/'
interim = '/home/mitch/Dropbox/data/mexico_enigh/interim2/'
dicts = '/home/mitch/school/data/mexico_enigh/dicts/'

In [55]:
cpidir = '/home/mitch/Dropbox/data/cpi/'
os.chdir(cpidir)
cpi = pd.read_csv('cpi_annual.csv')
cpi = cpi.rename(columns = {'time':'Y', 'PCPI_IX':'cpi'})
cpi = cpi.query('country == "MX"')
cpi = cpi.drop(columns=['Unnamed: 0', 'country'])

In [56]:
os.chdir(dicts)
rename_main = json_utils.load_json('rename_main.json')
rename_population = json_utils.load_json('rename_population.json')
rename_income = json_utils.load_json('rename_income.json')
hhm_ids = json_utils.load_json('hhm_ids.json')
education = json_utils.load_json('education.json')
print(rename_main)
print(rename_population)

{'folio': 'hhid', 'folioviv': 'hhid', 'ubica_geo': 'location', 'factor': 'weight', 'hog': 'weight', 'estrato': 'location_size', 'gastot': 'consumption_total', 'gascor': 'consumption_current', 'alimentos': 'consumption_food', 'vestido': 'consumption_clothing', 'vestido_c': 'consumption_clothing_c', 'salud': 'consumption_health', 'limpieza': 'consumption_house', 'transporte': 'consumption_transportation', 'educacion': 'consumption_education', 'personal': 'consumption_personal', 'ingtot': 'income_total', 'ingcor': 'income_current', 'trabajo': 'income_salary', 'negocio': 'income_business', 'rentas': 'income_rental', 'transfer': 'income_transfers', 'otros': 'income_other', 'pertot': 'financial_income', 'eromon': 'financial_expenditures', 'tam_hog': 'hh_size'}
{'folio': 'hhid', 'folioviv': 'hhid', 'paren': 'relationship_to_hhm', 'parentesco': 'relationship_to_hhm', 'sexo': 'sex', 'edad': 'age', 'tipo_esc': 'type_of_education', 'n_instr141': 'education', 'n_instr161': 'education', 'ed_formal'

In [57]:
year = '1994'
os.chdir(raw + year)
main = Dbf5('main.dbf').to_dataframe()
population = Dbf5('population.dbf').to_dataframe()

main.columns = [x.lower() for x in main.columns]
population.columns = [x.lower() for x in population.columns]

In [58]:
main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12815 entries, 0 to 12814
Data columns (total 49 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   folio       12815 non-null  object 
 1   ubica_geo   12815 non-null  object 
 2   hog         12815 non-null  float64
 3   estrato     12815 non-null  object 
 4   ingmon      12815 non-null  float64
 5   permon      12815 non-null  float64
 6   gasnom      12815 non-null  float64
 7   eronom      12815 non-null  float64
 8   ingcor      12815 non-null  float64
 9   gasmon      12815 non-null  float64
 10  gascor      12815 non-null  float64
 11  tam_hog     12815 non-null  int64  
 12  hombres     12815 non-null  int64  
 13  mujeres     12815 non-null  int64  
 14  tot_resi    12815 non-null  int64  
 15  tot_hom     12815 non-null  int64  
 16  tot_muj     12815 non-null  int64  
 17  ingtot      12815 non-null  float64
 18  pertot      12815 non-null  float64
 19  eromon      12815 non-nul

In [46]:
population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60353 entries, 0 to 60352
Data columns (total 37 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   folio       60353 non-null  object
 1   num_ren     60353 non-null  object
 2   parentesco  60353 non-null  object
 3   sexo        60353 non-null  object
 4   edad        60353 non-null  int64 
 5   per_ing     60353 non-null  object
 6   leer_esc    60353 non-null  object
 7   asis_esc_f  60353 non-null  object
 8   tipo_esc    60353 non-null  object
 9   ed_formal   60353 non-null  object
 10  ed_tecnica  60353 non-null  object
 11  trabajo     60353 non-null  object
 12  no_trabajo  60353 non-null  object
 13  n_empleos   60353 non-null  int64 
 14  ocupacion   60353 non-null  object
 15  oc_donde    60353 non-null  object
 16  rama        60353 non-null  object
 17  ra_donde    60353 non-null  object
 18  nom_empr    60353 non-null  object
 19  posicion    60353 non-null  object
 20  hrs_se

In [47]:
years = ['1994', '1996']
dfs = []

for year in years:
    print('Making year ' + year)
    os.chdir(raw + year)
    main = Dbf5('main.dbf').to_dataframe()
    population = Dbf5('population.dbf').to_dataframe()

    print('Number of houses', len(main))
    print('Number of people', len(population))

    main.columns = [x.lower() for x in main.columns]
    population.columns = [x.lower() for x in population.columns]
     
    main = main.rename(columns = rename_main)
    population = population.rename(columns = rename_population)

    keep_main = list(set(rename_main.values()) & set (main.columns))
    keep_population = list(set(rename_population.values()) & set (population.columns))

    main = main[keep_main]
    population = population[keep_population]

    population['relationship_to_hhm'] = population['relationship_to_hhm'].astype('int')
    population['is_household_head'] = population['relationship_to_hhm'].apply(lambda x : x in hhm_ids[year])

    population = population.query('is_household_head == True')
    print('Number of household heads', len(population))
    population['education'] = (population['education']
                               .astype(int)
                               .apply(lambda x : education[year][str(x)])
                               .astype(str))

    df = main.merge(population, on='hhid')

    # add year indicator
    df['Y'] = int(year)
    dfs.append(df)

Making year 1994
Number of houses 12815
Number of people 60353
Number of household heads 12815
Making year 1996
Number of houses 14042
Number of people 64916
Number of household heads 14042


In [48]:
# concatenate to build total dataset over each year 
df = pd.concat(dfs).reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26857 entries, 0 to 26856
Data columns (total 31 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   income_salary               26857 non-null  float64
 1   consumption_total           26857 non-null  float64
 2   income_business             26857 non-null  float64
 3   hh_size                     26857 non-null  int64  
 4   consumption_food            26857 non-null  float64
 5   consumption_health          26857 non-null  float64
 6   income_other                26857 non-null  float64
 7   hhid                        26857 non-null  object 
 8   location                    26857 non-null  object 
 9   income_current              26857 non-null  float64
 10  consumption_personal        26857 non-null  float64
 11  income_total                26857 non-null  float64
 12  weight                      26857 non-null  float64
 13  consumption_house           268

In [49]:
df['sex'] = df['sex'].replace({'1' : 'Male', '2':'Female'})
df.sex.value_counts(dropna=False)

Male      22899
Female     3958
Name: sex, dtype: int64

In [50]:
df = df.merge(cpi, on='Y', how='left')
for column in df.columns:
    if 'income' in column or 'consumption' in column:
        df[column]  = df[column] / df['cpi']
        df[column] = df[column] / df['hh_size']

AttributeError: 'DataFrame' object has no attribute 'estrato'

In [51]:
os.chdir(interim)
df.to_csv('main.csv', index=False)
